# Dietní problém v PuLP + duální úloha

V tomto zápisníku:

1. Zformulujeme **dietní problém** jako úlohu lineárního programování (LP).
2. Vyřešíme primární úlohu v PuLP.
3. Odvodíme **duální úlohu**.
4. Duální úlohu také vyřešíme v PuLP.
5. Porovnáme optimální hodnoty a ověříme tak **větu o silné dualitě**.


In [ ]:
!pip -q install pulp pandas numpy matplotlib

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pulp


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 43.3 MB/s eta 0:00:00


## Zadání problému

Cíl: minimalizovat cenu jedné porce přílohy v restauraci při splnění minimálních požadavků
na vitamín A, vitamín C a vlákninu.

Máme tři potraviny:

- Mrkev (syrová)
- Zelí (syrové)
- Okurka (kyselá)

Rozhodovací proměnné (kg na porci):

- $x_{\mathrm{carrot}}$  … kg syrové mrkve na porci,
- $x_{\mathrm{cabbage}}$ … kg syrového zelí na porci,
- $x_{\mathrm{cucumber}}$ … kg kyselé okurky na porci.

Účelová funkce – **minimální cena porce** (Kč):

$$
\min\; 15 x_{\mathrm{carrot}} + 10 x_{\mathrm{cabbage}} + 3 x_{\mathrm{cucumber}}.
$$

Omezení – **nutriční požadavky na porci**:

- vitamín A ≥ 0.5 mg,
- vitamín C ≥ 15 mg,
- vláknina ≥ 4 g.

Použijeme následující „hustoty živin“ (na 1 kg potraviny):

| potravina            | cena [Kč/kg] | vit. A [mg/kg] | vit. C [mg/kg] | vláknina [g/kg] |
|----------------------|-------------|----------------|----------------|-----------------|
| mrkev (raw)          | 15          | 35             | 60             | 30              |
| zelí (raw)           | 10          | 0.5            | 300            | 20              |
| okurka (pickled)     | 3           | 0.5            | 10             | 10              |

Formálně:

\begin{aligned}
\min\quad
 & 15x_{\mathrm{carrot}} + 10x_{\mathrm{cabbage}} + 3x_{\mathrm{cucumber}} \\
\text{při}\quad
 & 35x_{\mathrm{carrot}} + 0.5x_{\mathrm{cabbage}} + 0.5x_{\mathrm{cucumber}} \ge 0.5
        &&\text{(vitamín A)}\\
 & 60x_{\mathrm{carrot}} + 300x_{\mathrm{cabbage}} + 10x_{\mathrm{cucumber}} \ge 15
        &&\text{(vitamín C)}\\
 & 30x_{\mathrm{carrot}} + 20x_{\mathrm{cabbage}} + 10x_{\mathrm{cucumber}} \ge 4
        &&\text{(vláknina)}\\
 & x_{\mathrm{carrot}},x_{\mathrm{cabbage}},x_{\mathrm{cucumber}} \ge 0.
\end{aligned}


In [ ]:
# Data v Pandas tabulce

foods = ["Carrot_raw", "Cabbage_raw", "Cucumber_pickled"]

data = pd.DataFrame(
    {
        "price_KC_per_kg":        [15.0,   10.0,   3.0],
        "vitamin_A_mg_per_kg":    [35.0,    0.5,   0.5],
        "vitamin_C_mg_per_kg":    [60.0,  300.0,  10.0],
        "fiber_g_per_kg":         [30.0,   20.0,  10.0],
    },
    index=foods,
)

requirements = {
    "vitamin_A_mg": 0.5,
    "vitamin_C_mg": 15.0,
    "fiber_g":      4.0,
}

display(data.style.format(precision=3).set_caption("Food data (per kg)"))
pd.Series(requirements, name="Min per serving").to_frame()


,price_KC_per_kg,vitamin_A_mg_per_kg,vitamin_C_mg_per_kg,fiber_g_per_kg
Carrot_raw,15.000,35.000,60.000,30.000
Cabbage_raw,10.000,0.500,300.000,20.000
Cucumber_pickled,3.000,0.500,10.000,10.000


,Min per serving
vitamin_A_mg,0.5
vitamin_C_mg,15.0
fiber_g,4.0


## Formulace a řešení primární úlohy v PuLP


In [ ]:
def solve_diet_primal_pulp(data: pd.DataFrame, req: dict, msg: bool = False):
    # LP model: minimalizace
    prob = pulp.LpProblem("Diet_primal", pulp.LpMinimize)

    # Rozhodovací proměnné: kg na porci
    x = {f: pulp.LpVariable(f"x_{f}", lowBound=0) for f in data.index}

    # Účelová funkce
    prob += pulp.lpSum(data.loc[f, "price_KC_per_kg"] * x[f] for f in data.index)

    # Omezení – vitamín A
    prob += (
        pulp.lpSum(data.loc[f, "vitamin_A_mg_per_kg"] * x[f] for f in data.index)
        >= req["vitamin_A_mg"],
        "vitamin_A",
    )

    # Omezení – vitamín C
    prob += (
        pulp.lpSum(data.loc[f, "vitamin_C_mg_per_kg"] * x[f] for f in data.index)
        >= req["vitamin_C_mg"],
        "vitamin_C",
    )

    # Omezení – vláknina
    prob += (
        pulp.lpSum(data.loc[f, "fiber_g_per_kg"] * x[f] for f in data.index)
        >= req["fiber_g"],
        "fiber",
    )

    # Řešení (CBC je výchozí solver v PuLP)
    prob.solve(pulp.PULP_CBC_CMD(msg=msg))

    status = pulp.LpStatus[prob.status]
    x_kg = {f: v.value() for f, v in x.items()}
    cost = pulp.value(prob.objective)

    return prob, status, x_kg, cost


primal_prob, primal_status, primal_x, primal_cost = solve_diet_primal_pulp(
    data, requirements, msg=False
)

print("Status:", primal_status)
print("Optimální cena [Kč]:", round(primal_cost, 4))
print("\nOptimální množství (kg na porci):")
pd.Series(primal_x, name="kg per serving").to_frame()


Status: Optimal
Optimální cena [Kč]: 1.4102

Optimální množství (kg na porci):


,kg per serving
Carrot_raw,0.009526
Cabbage_raw,0.038265
Cucumber_pickled,0.294891


## Odvození duální úlohy

Zapíšeme primární úlohu ve zhuštěném tvaru

\begin{aligned}
\min\quad     & c^\mathsf{T} x \\
\text{při}\quad
 & Ax \ge b, \\
 & x \ge 0,
\end{aligned}

kde

- $x \in \mathbb{R}^3$ – množství jednotlivých potravin,
- $c \in \mathbb{R}^3$ – ceny za 1 kg,
- $A \in \mathbb{R}^{3\times 3}$ – matice „hustot živin“ (řádky = živiny, sloupce = potraviny),
- $b \in \mathbb{R}^3$ – minimální požadované množství živin.

Pro takto zadanou **minimalizační** úlohu s omezeními typu „$\ge$“ a podmínkou $x \ge 0$ má
duální úloha tvar

\begin{aligned}
\max\quad     & b^\mathsf{T} y \\
\text{při}\quad
 & A^\mathsf{T} y \le c, \\
 & y \ge 0.
\end{aligned}

Duální proměnné $y$ zde můžeme interpretovat jako **stínové ceny** (Kč na jednotku živiny):

- $y_A$ … cena za 1 mg vitamínu A,
- $y_C$ … cena za 1 mg vitamínu C,
- $y_F$ … cena za 1 g vlákniny.

### Konkrétní tvar duálního LP

V našem případě:

$$
b =
\begin{pmatrix}
0.5 \\ 15 \\ 4
\end{pmatrix},
\quad
y =
\begin{pmatrix}
y_A \\ y_C \\ y_F
\end{pmatrix},
$$

a matice $A$ (řádky A,C,F; sloupce mrkev, zelí, okurka) je

$$
A =
\begin{pmatrix}
35 & 0.5 & 0.5 \\
60 & 300 & 10 \\
30 & 20 & 10
\end{pmatrix}.
$$

Duální úloha tedy vychází:

\begin{aligned}
\max\quad
 & 0.5 y_A + 15 y_C + 4 y_F \\
\text{při}\quad
 & 35 y_A + 60 y_C + 30 y_F \le 15 &&\text{(mrkev)}\\
 & 0.5 y_A + 300 y_C + 20 y_F \le 10 &&\text{(zelí)}\\
 & 0.5 y_A + 10 y_C + 10 y_F \le 3 &&\text{(okurka)}\\
 & y_A, y_C, y_F \ge 0.
\end{aligned}


Intuitivně:

- účelová funkce „prodává“ minimálně potřebné množství živin za ceny $y_A,y_C,y_F$,
- nerovnosti $A^\mathsf{T} y \le c$ říkají, že **cena živin obsažených v 1 kg konkrétní potraviny
  nesmí přesáhnout cenu 1 kg této potraviny** (jinak by se vyplatilo kupovat živiny přes danou potravinu
  a prodávat je samostatně).


In [ ]:
def solve_diet_dual_pulp(data: pd.DataFrame, req: dict, msg: bool = False):
    # LP model: maximalizace
    prob = pulp.LpProblem("Diet_dual", pulp.LpMaximize)

    # Duální proměnné – stínové ceny živin
    yA = pulp.LpVariable("y_A", lowBound=0)
    yC = pulp.LpVariable("y_C", lowBound=0)
    yF = pulp.LpVariable("y_F", lowBound=0)

    y = {"vitamin_A_mg": yA, "vitamin_C_mg": yC, "fiber_g": yF}

    # Účelová funkce: b^T y
    prob += (
        req["vitamin_A_mg"] * yA
        + req["vitamin_C_mg"] * yC
        + req["fiber_g"] * yF
    )

    # Omezení: A^T y <= c  (po sloupcích, tj. po potravinách)
    for f in data.index:
        prob += (
            data.loc[f, "vitamin_A_mg_per_kg"] * yA
            + data.loc[f, "vitamin_C_mg_per_kg"] * yC
            + data.loc[f, "fiber_g_per_kg"] * yF
            <= data.loc[f, "price_KC_per_kg"],
            f"price_bound_{f}",
        )

    # Řešení
    prob.solve(pulp.PULP_CBC_CMD(msg=msg))

    status = pulp.LpStatus[prob.status]
    y_val = {name: var.value() for name, var in y.items()}
    value = pulp.value(prob.objective)

    return prob, status, y_val, value


dual_prob, dual_status, dual_y, dual_value = solve_diet_dual_pulp(
    data, requirements, msg=False
)

print("Status:", dual_status)
print("Optimální hodnota duálu [Kč]:", round(dual_value, 4))
print("\nOptimální stínové ceny živin:")
pd.Series(dual_y, name="Kč per unit").to_frame()


Status: Optimal
Optimální hodnota duálu [Kč]: 1.4102

Optimální stínové ceny živin:


,Kč per unit
vitamin_A_mg,0.166046
vitamin_C_mg,0.014582
fiber_g,0.277115


## Ověření silné duality

Silná dualita říká, že pro lineární programy, které mají optimální řešení
a splňují přirozené předpoklady (např. je přípustná množina neprázdná a omezená),
platí:

$$
\min \text{(primár)} = \max \text{(duál)}.
$$

Porovnáme numericky optimální hodnoty obou úloh.


In [ ]:
print("Primár – optimální cena [Kč]:", round(primal_cost, 6))
print("Duál   – optimální hodnota [Kč]:", round(dual_value, 6))
print("Rozdíl |P* - D*|:", abs(primal_cost - dual_value))


Primár – optimální cena [Kč]: 1.410218
Duál   – optimální hodnota [Kč]: 1.410218
Rozdíl |P* - D*|: 1.2999999743357193e-08
